In [10]:
from google.cloud import bigquery
import pandas as pd
from datetime import date,datetime,timedelta,timezone
from google.cloud import bigquery
from google.oauth2 import service_account
from dotenv import dotenv_values
from datetime import datetime ,timezone
import sys

In [11]:
load_from='2023-07-22'

is_py=False
#modelType='multi'
modelType='binary'


projectId='smart-data-ml'
dataset_id='SMartML'


imported_at_utc=datetime.now(timezone.utc)
imported_at= datetime.now()

today_str=imported_at_utc.strftime("%Y-%m-%d")
predict_datetime = datetime.strptime(today_str,"%Y-%m-%d")

print(f"Get prediction as of {predict_datetime}(UTC) at {imported_at}(Asian/BKK)-{imported_at_utc}(UTC)")

env_path=r'D:\PythonDev\MyQuantFinProject\SMart-AI\.env'
config = dotenv_values(dotenv_path=env_path)

Get prediction as of 2023-07-23 00:00:00(UTC) at 2023-07-23 16:22:34.765737(Asian/BKK)-2023-07-23 09:22:34.764785+00:00(UTC)


In [12]:
if is_py:
    press_Y=''
    ok=False

    if len(sys.argv) > 1:
        modelType=sys.argv[1]
        ok=True 
    else:
        print("Enter the following input: ")
        modelType = input("Type (multi,binary): ")
        load_from = input("Load From (2023-07-20) : ")

        print(f"Confirm to {modelType} and {datetime.strptime(load_from,'%Y-%m-%d')}")
        press_Y=input(f"Press Y=True But any key=False : ") 
        if press_Y=='Y':
         ok=True
        
    if modelType not in ['binary','multi']:
        raise Exception("model type must be binary or multi")

    if ok==False:
        print("No any action")
        quit()
    

In [13]:
if modelType=='multi':
  
 model_version='model_v2_t150723'
 table_id = f"{projectId}.{dataset_id}.new_result_prediction_incident"
 sm_table="app_prediction_ml_severity_incident"   
 if load_from!='':
    sql=f""" select id,predict_severity,prediction_item_date from {table_id} 
    where Date(prediction_datetime)>='{load_from}' """
 else:
    sql=f"""
    select id,predict_severity,prediction_item_date from {table_id} 
     where Date(prediction_datetime)='{today_str}' """   
    
elif modelType=='binary':
 model_version='tuned_binary_xgb_tf_model_v1_t150723'    
 table_id = f"{projectId}.{dataset_id}.new2_result_binary_prediction_incident"   
 sm_table="app_prediction_ml2_everity_incident"      
 if load_from!='':
    sql=f""" select id,label_binary_severity,prediction_item_date from {table_id} 
    where Date(prediction_datetime)>='{load_from}' """
 else:
    sql=f"""
    select id,label_binary_severity,prediction_item_date from {table_id} 
    where Date(prediction_datetime)='{today_str}' """   
       

print(f"{modelType} : {model_version}")
print(f"{table_id } to { sm_table}")
print(sql)
    


binary : tuned_binary_xgb_tf_model_v1_t150723
smart-data-ml.SMartML.new2_result_binary_prediction_incident to app_prediction_ml2_everity_incident
 select id,label_binary_severity,prediction_item_date from smart-data-ml.SMartML.new2_result_binary_prediction_incident 
    where Date(prediction_datetime)>='2022-07-22' 


In [14]:
credentials = service_account.Credentials.from_service_account_file(r'C:\Windows\smart-data-ml-91b6f6204773.json')
client = bigquery.Client(credentials= credentials,project=projectId)

In [15]:
def load_data_bq(sql:str):

 query_result=client.query(sql)
 df_all=query_result.to_dataframe()
 return df_all



In [16]:
df=load_data_bq(sql)
if len(df)>0:
    df.columns=['incident_id','severity_label','prediction_at']
else:
    print("No record")
    quit()

In [17]:
# load json file
if modelType=='multi':
    map_severity_to_class={'Cosmetic':0,'Minor': 1, "Major": 2, "Critical": 3}
    revert_class_to_severity = {v: k for k, v in map_severity_to_class.items()}
    print(f"Map severity Name to LabelCode: {str(revert_class_to_severity)}")
    df['severity_name']=df['severity_label'].map(revert_class_to_severity)
elif modelType=='binary':
    map_severity_to_class={1:'Critical',0: 'Normal'}
    print(f"Map severity Name to LabelCode: {str(map_severity_to_class)}")
    df['severity_name']=df['severity_label'].map(map_severity_to_class)



df['imported_at']=imported_at
df['model_version']=model_version

print(df.info())
df

Map severity Name to LabelCode: {1: 'Critical', 0: 'Normal'}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   incident_id     53 non-null     Int64         
 1   severity_label  53 non-null     Int64         
 2   prediction_at   53 non-null     datetime64[ns]
 3   severity_name   53 non-null     object        
 4   imported_at     53 non-null     datetime64[ns]
 5   model_version   53 non-null     object        
dtypes: Int64(2), datetime64[ns](2), object(2)
memory usage: 2.7+ KB
None


,incident_id,severity_label,prediction_at,severity_name,imported_at,model_version
0,3213,0,2023-07-22,Normal,2023-07-23 16:22:34.765737,tuned_binary_xgb_tf_model_v1_t150723
1,3198,1,2023-07-22,Critical,2023-07-23 16:22:34.765737,tuned_binary_xgb_tf_model_v1_t150723
2,3154,0,2023-07-21,Normal,2023-07-23 16:22:34.765737,tuned_binary_xgb_tf_model_v1_t150723
3,3188,0,2023-07-21,Normal,2023-07-23 16:22:34.765737,tuned_binary_xgb_tf_model_v1_t150723
4,3187,0,2023-07-21,Normal,2023-07-23 16:22:34.765737,tuned_binary_xgb_tf_model_v1_t150723
5,3169,0,2023-07-21,Normal,2023-07-23 16:22:34.765737,tuned_binary_xgb_tf_model_v1_t150723
6,3184,0,2023-07-21,Normal,2023-07-23 16:22:34.765737,tuned_binary_xgb_tf_model_v1_t150723
7,3165,0,2023-07-21,Normal,2023-07-23 16:22:34.765737,tuned_binary_xgb_tf_model_v1_t150723
8,3196,0,2023-07-21,Normal,2023-07-23 16:22:34.765737,tuned_binary_xgb_tf_model_v1_t150723
9,3172,0,2023-07-21,Normal,2023-07-23 16:22:34.765737,tuned_binary_xgb_tf_model_v1_t150723


In [18]:
import psycopg2
import psycopg2.extras as extras

def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error

def add_data_values(df, table,conn):

    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    #print(query)
    #return query,tuples
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        raise error
        return 0
    
    return 1
    cursor.close()

result=add_data_values(df,sm_table,get_postgres_conn())

if  result==1:
    print(f"{len(df.index)} items have been imported to {sm_table} successfully.")

53 items have been imported to app_prediction_ml2_everity_incident successfully.
